In [79]:
# Name: Abdullahi Sani Shuaibu
# ID: 202303370
# Project: Embedded Markov Proces based Model for Performance Analysis of IDPS 
# Course: COE 520 (Queuing Theory and Network Applications)
# Instructor: Dr. Mohammad Abdurazzak Felemban
# Term: 231
# Date: 6 October, 2023

### Constants

In [80]:
L = 3 # L-1 is queue length
L_list = [5, 10, 25, 50, 100]
p = 0.3 # probability of early decision
r = 1 - p # probability of content checking

### Settings

In [81]:
micro = 10**-6;
settings  = {
    1: {'alpha': (1/0.5)*micro, 'mu': (1/4)*micro},
    2: {'alpha': (1/0.5)*micro, 'mu': (1/8)*micro},
    3: {'alpha': (1/0.5)*micro, 'mu': (1/12)*micro},
    4: {'alpha': (1/5.0)*micro, 'mu': (1/12)*micro},
    }

### Coefficients

In [82]:
w01 = w02 = 1
w11 = lambda setting, lam: ((setting['alpha'] + lam)*lam)/((setting['alpha'] + p*lam)*setting['mu'])
w12 = lambda setting, lam: (lam - p*lam)/(setting['alpha'] + p*lam)

def wn1(n, setting, lam):
    """ for 2 <= n < L """
    if n == 0:
        return w01
    if n == 1:
        return w11(setting, lam)
    return (((setting['alpha'] + lam)*(lam + setting['mu']))/((p*lam + setting['alpha'])*setting['mu']))*wn1(n-1, setting, lam) - ((lam + setting['alpha'])/(lam*p + setting['alpha']))*wn1(n-2, setting, lam) - ((lam*setting['alpha'])/((lam*p + setting['alpha'])*setting['mu']))*wn2(n-1, setting, lam)

def wn2(n, setting, lam):
    """ for 2 <= n < L """
    if n == 0:
        return w02
    if n == 1:
        return w12(setting, lam)
    return (lam/(lam + setting['alpha']))*wn2(n-1, setting, lam) + (((1 - p)*setting['mu'])/(lam + setting['alpha']))*wn1(n, setting, lam)

# print(wn1(15, settings[1], 5))
# print(wn2(15, settings[1], 5))

### Probabilities

In [83]:
q00 = lambda setting, lam: 1/(1 + sum([wn1(n, setting, lam) + wn2(n, setting, lam) for n in range(1, L+1)]))

def qn1(n, setting, lam, L=L):
    if n == L and L > 1:
        return ((lam)/(setting['mu']))*wn1(L-1, setting, lam)*q00(setting, lam)
    if n == L and L == 1:
        return ((lam)/(setting['mu']))*q00(setting, lam)
    return wn1(n, setting, lam)*q00(setting, lam)
    
def qn2(n, setting, lam, L=L):
    if n == L and L > 1:
        return ((lam)/(setting['alpha']))*(wn2(L-1, setting, lam) + (1-p)*wn1(L-1, setting, lam))*q00(setting, lam)
    if n == L and L == 1:
        return ((1-p)*lam)/(setting['alpha'])*q00(setting, lam)
    return wn2(n, setting, lam)*q00(setting, lam)
    
# print('q00: ', q00(settings[2], 5))
# print('q21: ', qn1(2, settings[2], 5))
# print('q22: ', qn2(2, settings[2], 5))

## Performance Metrices

#### Throughput

In [84]:
def throughput(setting, lam, L=L):
    return p*setting['mu']*sum([qn1(n, setting, lam) for n in range(1, L+1)]) + setting['alpha']*sum([qn2(n, setting, lam) for n in range(1, L+1)])

#### Probability of Packet Loss

In [85]:
def ppl(setting, lam, L=L):
    return qn1(L, setting, lam) + qn2(L, setting, lam)

#### Average Number of Packets in the System

In [86]:
def avgn(setting, lam, L=L):
    return sum([n*(qn1(n, setting, lam) + qn2(n, setting, lam)) for n in range(1, L+1)])

#### Average Packet Time in the System

In [87]:
def avgt(setting, lam, L=L):
    return avgn(setting, lam, L)/(throughput(setting, lam, L) + 0.00000000000000000000001)

#### Average Service Time of the Two Stages

In [88]:
def avgst(setting):
    return (1/setting['mu']) + ((1-p)/setting['alpha'])

#### Average Packet Queue Time

In [89]:
def avgqt(setting, lam, L=L):
    return avgt(setting, lam, L) - avgst(setting)

## Generate Data

In [90]:
import csv
import os
import json

# for k, v in settings.items():
#     print(k, v)
def data_gen(type='settings'):
    data_dir = os.path.join('data', type)
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    csv_path = os.path.join(data_dir, type + '.csv')
    json_path = os.path.join(data_dir, type + '.json')
    csv_file= open(csv_path, mode='w')
    json_file= open(json_path, mode='w')
        
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    lams = [lam for lam in range(0, 500, 25)]
    header = [str(lam) for lam in lams]

    if type == 'settings':
        data = {}
        for setting in settings:
            data[setting] = {}
        for setting in settings:
            for lam in lams:
                this_point = {}
                this_point['throughput'] = throughput(settings[setting], lam)
                this_point['ppl'] = ppl(settings[setting], lam)
                this_point['delay'] = avgt(settings[setting], lam)
                data[setting].update({lam: this_point})

            csv_writer.writerow([f'setting: {setting}'])
            csv_writer.writerow(header)
            throughputs = []
            ppls = []
            delays = []
            for lam in data[setting]:
                throughputs.append(data[setting][lam]['throughput'])
                ppls.append(data[setting][lam]['ppl'])
                delays.append(data[setting][lam]['delay'])
            csv_writer.writerow(throughputs)
            csv_writer.writerow(ppls)
            csv_writer.writerow(delays)
        json.dump(data, json_file)

    if type == 'population':
        data = {}
        for population in L_list:
            data[population] = {}
        for population in L_list:
            for lam in lams:
                this_point = {}
                this_point['throughput'] = throughput(settings[1], lam, L=population)
                this_point['ppl'] = ppl(settings[1], lam, L=population)
                this_point['delay'] = avgt(settings[1], lam, L=population)
                data[population].update({lam: this_point})

            csv_writer.writerow([f'population: {population}'])
            csv_writer.writerow(header)
            throughputs = []
            ppls = []
            delays = []
            for lam in data[population]:
                throughputs.append(data[population][lam]['throughput'])
                ppls.append(data[population][lam]['ppl'])
                delays.append(data[population][lam]['delay'])
            csv_writer.writerow(throughputs)
            csv_writer.writerow(ppls)
            csv_writer.writerow(delays)
        json.dump(data, json_file)




data_gen(type='population')

tp:  0.0
L:  5
tp:  8333331886.111504
L:  5
tp:  33333330438.889294
L:  5
tp:  74999995658.33374
L:  5
tp:  133333327544.44484
L:  5
tp:  208333326097.2226
L:  5
tp:  299999991316.66705
L:  5
tp:  408333323202.7781
L:  5
tp:  533333321755.5559
L:  5
tp:  674999986975.0005
L:  5
tp:  833333318861.1118
L:  5
tp:  1008333317413.8896
L:  5
tp:  1199999982633.3342
L:  5
tp:  1408333314519.4453
L:  5
tp:  1633333313072.223
L:  5
tp:  1874999978291.6677
L:  5
tp:  2133333310177.7786
L:  5
tp:  2408333308730.5557
L:  5
tp:  2699999973950.0005
L:  5
tp:  3008333305836.113
L:  5
tp:  2.1276595744680852e-07
L:  10


tp:  3.4293516563811696e+52
L:  10
tp:  4.3895724400556976e+54
L:  10
tp:  7.499997357500625e+55
L:  10
tp:  5.618654208000265e+56
L:  10
tp:  2.6791832470636926e+57
L:  10
tp:  9.599998308800193e+57
L:  10
tp:  2.8242211098937555e+58
L:  10
tp:  7.191878336467132e+58
L:  10
tp:  1.640249807361766e+59
L:  10
tp:  3.429354918724309e+59
L:  10
tp:  6.682842636303214e+59
L:  10
tp:  1.2287998917632076e+60
L:  10
tp:  2.1518694066551186e+60
L:  10
tp:  3.61500329359674e+60
L:  10
tp:  5.859374587109405e+60
L:  10
tp:  9.205604878823179e+60
L:  10
tp:  1.4071970078415315e+61
L:  10
tp:  2.0995198767115258e+61
L:  10
tp:  3.0654035983282127e+61
L:  10


KeyboardInterrupt: 

## Visualizations for Security Levels (L=25, p=0.3)

### Throughput

In [ ]:
a = {'a': 2}
a.update({'b': 4})
a

{'a': 2, 'b': 4}

### Packet Loss Ratio

### Average Packet Time in System

## Visualizations for System Population (Security Level 1, p=0.3)

### Throughput

### Packet Loss Ratio

### Average Packet Time in System